# Задание 4 - Перенос обучения (transfer learning) и тонкая настройка (fine-tuning)

Одной из важнейшних техник в тренировке сетей - использовать заранее натренированные веса на более общей задачи в качестве начальной точки, а потом "дотренировать" их на конкретной.

Такой подход и убыстряет обучение, и позволяет тренировать эффективные модели на маленьких наборах данных.

В этом упражнении мы натренируем классификатор, который отличает хотдоги от не хотдогов!  
(более подробно - https://www.youtube.com/watch?v=ACmydtFDTGs)

Это задание требует доступа к GPU, поэтому его можно выполнять либо на компьютере с GPU от NVidia, либо в [Google Colab](https://colab.research.google.com/).

In [1]:
import json
import os
import csv
import urllib
from io import BytesIO
from PIL import Image

from socket import timeout

#from google.colab import files

import re
from skimage import io, transform

!pip3 install -q torch torchvision
#!pip3 install -q Pillow==4.0.0
!pip3 install -q Pillow

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


Сначала давайте скачаем данные с картинками. Это сделает код в следующей ячейке. Данные будут разделены на две части. На обучающей выборке, которая будет храниться в папке **train_kaggle**, мы будем строить наши модели, а на тестовой выборке **test_kaggle** будем предсказывать класс, к которому относится фотография (хотдог или нет).

### Если вы в Google Colab!

В нем можно запускать ноутбуки с доступом к GPU. Они не очень быстрые, зато бесплатные!
Каждый ноутбук получает свой собственный environment c доступным диском итд.

Через 90 минут отсуствия активности этот environment пропадает со всеми данными.
Поэтому нам придется скачивать данные каждый раз.

In [2]:
# Download train data
!wget -nc "https://storage.googleapis.com/dlcourse_ai/train.zip"
!unzip -qn "train.zip"

train_folder = "train_kaggle/"
# Count number of files in the train folder, should be 4603
print('Number of files in the train folder', len(os.listdir(train_folder)))

# Download test data
!wget  -nc "https://storage.googleapis.com/dlcourse_ai/test.zip"
!unzip -qn "test.zip"

test_folder = "test_kaggle/"
# Count number of files in the test folder, should be 1150
print('Number of files in the test folder', len(os.listdir(test_folder)))

--2020-07-30 12:13:01--  https://storage.googleapis.com/dlcourse_ai/train.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.21.240, 172.217.23.176, 172.217.23.144, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.21.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 562348083 (536M) [application/zip]
Saving to: 'train.zip'

train.zip           100%[===================>] 536.30M  51.3MB/s    in 13s     

2020-07-30 12:13:15 (40.4 MB/s) - 'train.zip' saved [562348083/562348083]

Number of files in the train folder 4603
--2020-07-30 12:13:19--  https://storage.googleapis.com/dlcourse_ai/test.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.21.208, 172.217.16.208, 172.217.22.80, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.21.208|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 140788786 (134M) [application/zip]
Saving to: 'test.zip'

test

In [56]:
import torch
from torchvision import models
from torch.utils.data import Dataset, SubsetRandomSampler
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from torch.utils.data.sampler import Sampler
import torch.nn.functional as tf
import sklearn.metrics as metrics

from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

device = torch.device("cuda:0") # Let's make sure GPU is available!

# Имплементируем свой Dataset для загрузки данных

В этом задании мы реализуем свой собственный класс Dataset для загрузки данных. Его цель - загрузить данные с диска и выдать по ним тензор с входом сети, меткой и идентификатором картинки (так будет проще подготовить сабмит для kaggle на тестовых данных).

Вот ссылка, где хорошо объясняется как это делать на примере: https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

Ваш Dataset должен в качестве количества сэмплов выдать количество файлов в папке и уметь выдавать кортеж из сэмпла, метки по индексу и названия файла.
Если название файла начинается со слов 'frankfurter', 'chili-dog' или 'hotdog' - метка положительная. Иначе отрицательная (ноль).

И не забудьте поддержать возможность трансформации входа (аргумент `transforms`), она нам понадобится!

In [ ]:
import torchvision.transforms as tvtf

class HotdogOrNotDataset(Dataset):
    def __init__(self, folder, transform=None):
        self.transform = transform
        self.folder = folder
        
        # TODO: Your code here!
        
    def __len__(self):
        return len(os.listdir(self.folder))
    
    def __getitem__(self, index):        
        # TODO Implement getting item by index
        # Hint: os.path.join is helpful!
        img_name = os.listdir(self.folder)[index]
        img_path = os.path.join(self.folder, img_name
                                )
        image = io.imread(img_path)
        #print(re.match(r'frankfurter|chili-dog|hotdog', img_name), img_name)
        if re.match(r'frankfurter|chili-dog|hotdog', img_name):
            y = 1
        else:
            y = 0
        #plt.imshow(image)
        imagePIL = tvtf.ToPILImage()(image)

        if self.transform:

            imagePIL = self.transform(imagePIL)

        return imagePIL, y, img_name

def visualize_samples(dataset, indices, title=None, count=10, labels=None):
    # visualize random 10 samples
    fig1 = plt.figure(figsize=(count*3,3))
    display_indices = indices[:count]
    if title:
        fig1.suptitle("%s %s/%s" % (title, len(display_indices), len(indices)))        
    for i, index in enumerate(display_indices):    
        x, y, _ = dataset[index]
        ax = fig1.add_subplot(1,count,i+1)
        if labels:
          ax.set_title(labels[i])  
        else:
          ax.set_title("Label: %s" % y)
        ax.imshow(x)
        plt.grid(False)
        plt.axis('off')   
    
orig_dataset = HotdogOrNotDataset(train_folder)
indices = np.random.choice(np.arange(len(orig_dataset)), 7, replace=False)

visualize_samples(orig_dataset, indices, "Samples")

In [ ]:
# Let's make sure transforms work!
dataset = HotdogOrNotDataset(train_folder, transform=transforms.RandomVerticalFlip(0.9))

visualize_samples(dataset, indices, "Samples with flip - a lot should be flipped!")

# Создаем Dataset для тренировки

И разделяем его на train и validation.
На train будем обучать модель, на validation проверять ее качество, а соревнование Kaggle In-Class проведем на фотографиях из папки test_kaggle.

In [66]:
# First, lets load the dataset
train_dataset = HotdogOrNotDataset(train_folder, 
                       transform=transforms.Compose([
                           transforms.ColorJitter(hue=.50, saturation=.50, 
                                                  brightness = .50, contrast = .50),
                           transforms.RandomHorizontalFlip(),
                           transforms.RandomVerticalFlip(),
                           #transforms.RandomCrop((224, 224)),
                           transforms.RandomRotation(50, resample=Image.BILINEAR),
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           # Use mean and std for pretrained models
                           # https://pytorch.org/docs/stable/torchvision/models.html
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])                         
                       ])
                      )
val_dataset = HotdogOrNotDataset(train_folder, 
                       transform=transforms.Compose([
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           # Use mean and std for pretrained models
                           # https://pytorch.org/docs/stable/torchvision/models.html
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])                         
                       ])
                      )
test_dataset = HotdogOrNotDataset(test_folder, 
                       transform=transforms.Compose([
                           transforms.Resize((224, 224)),
                           transforms.ToTensor(),
                           # Use mean and std for pretrained models
                           # https://pytorch.org/docs/stable/torchvision/models.html
                           transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])                         
                       ])
                      )

In [67]:
batch_size = 32

data_size = len(dataset)
validation_fraction = .2


val_split = int(np.floor((validation_fraction) * data_size))
indices = list(range(data_size))
np.random.seed(42)
np.random.shuffle(indices)

val_indices, train_indices = indices[:val_split], indices[val_split:]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size,
                                         sampler=val_sampler)
# Notice that we create test data loader in a different way. We don't have the labels.
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size)

Наши обычные функции для тренировки

In [60]:
def train_model(model, train_loader, val_loader, loss, optimizer, scheduler=None,  num_epochs=10):    
    loss_history = []
    train_history = []
    val_history = []
    for epoch in range(num_epochs):
        model.train() # Enter train mode
        
        loss_accum = 0
        correct_samples = 0
        total_samples = 0
        for i_step, (x, y,_) in enumerate(train_loader):
          
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)    
            loss_value = loss(prediction, y_gpu)
            optimizer.zero_grad()
            loss_value.backward()
            optimizer.step()
            
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y.shape[0]
            
            loss_accum += loss_value.item()
            
            del x_gpu, y_gpu, prediction, loss_value, indices
        ave_loss = loss_accum / i_step
        train_accuracy = float(correct_samples) / total_samples
        val_accuracy, val_loss, val_f1 = compute_accuracy(model, val_loader)
        if scheduler:
          scheduler.step(val_loss)
          #scheduler.step()
        loss_history.append(float(ave_loss))
        train_history.append(train_accuracy)
        val_history.append(val_accuracy)

        
        print("Epoch: %d, Average loss: %f, Train accuracy: %f, Val accuracy: %f, Val F1: %f" % (epoch, ave_loss, train_accuracy, val_accuracy, val_f1))
        
    return loss_history, train_history, val_history
        
from scipy.special import softmax
    
def compute_accuracy(model, loader):
    """
    Computes accuracy on the dataset wrapped in a loader
    
    Returns: accuracy as a float value between 0 and 1
    """
    model.eval() # Evaluation mode
    # TODO: Implement the inference of the model on all of the batches from loader,
    #       and compute the overall accuracy.
    # Hint: PyTorch has the argmax function!
    correct_samples = 0
    total_samples = 0
    loss_accum = 0
    predictions = []
    ground_truth = []
    with torch.no_grad():
        for i_step, (x, y, _) in enumerate(loader):
            #print(x.shape)
            x_gpu = x.to(device)
            y_gpu = y.to(device)
            prediction = model(x_gpu)
            predictions.extend(torch.argmax(tf.softmax(prediction, dim=0), dim=1).cpu().detach().numpy()) 
            ground_truth.extend(y)
            _, indices = torch.max(prediction, 1)
            correct_samples += torch.sum(indices == y_gpu)
            total_samples += y_gpu.shape[0]
            loss_value = loss(prediction.cpu(), y)
            loss_accum += loss_value
            #loss_accum += 0
            del x_gpu, y_gpu, prediction, indices
        accuracy = float(correct_samples) / total_samples
        precision, recall, f1 = binary_classification_metrics(predictions, ground_truth)
    return accuracy, loss_accum, f1
    #return accuracy, loss_accum
    # Don't forget to move the data to device before running it through the model!


In [9]:
from torch.utils.data.sampler import Sampler
import torch.nn.functional as tf

class SubsetSampler(Sampler):
    r"""Samples elements with given indices sequentially

    Arguments:
        data_source (Dataset): dataset to sample from
        indices (ndarray): indices of the samples to take
    """

    def __init__(self, indices):
        self.indices = indices

    def __iter__(self):
        return (self.indices[i] for i in range(len(self.indices)))

    def __len__(self):
        return len(self.indices)
    
    
def evaluate_model(model, dataset, indices):
    """
    Computes predictions and ground truth labels for the indices of the dataset
    
    Returns: 
    predictions: np array of booleans of model predictions
    grount_truth: np array of boolean of actual labels of the dataset
    """

    val_sampler = SubsetSampler(indices)

    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, 
                                           sampler=val_sampler)

    model.eval() # Evaluation mode
    
    # TODO: Evaluate model on the list of indices and capture predictions
    # and ground truth labels
    # Hint: SubsetSampler above could be useful!
    
    predictions = []
    ground_truth = []
    for i_step, (x, y, _) in enumerate(loader):
        #print(i_step)
        x_gpu = x.to(device)
        prediction = model(x_gpu) 
        _, indices = torch.max(prediction, 1)
        ground_truth.extend(y.detach().numpy())
        predictions.extend(torch.argmax(tf.softmax(prediction, dim=0), dim=1).cpu().detach().numpy())
    
    return predictions, ground_truth

import sklearn.metrics as metrics

def binary_classification_metrics(prediction, ground_truth):
    # TODO: Implement this function!
    # We did this already it in the assignment1
    precision = metrics.precision_score(ground_truth, prediction)
    recall = metrics.recall_score(ground_truth, prediction)
    f1 = metrics.f1_score(ground_truth, prediction)
    return precision, recall, f1


# Использование заранее натренированной сети (pretrained network)

Чаще всего в качестве заранее натренированной сети используется сеть, натренированная на данных ImageNet с 1M изображений и 1000 классами.

PyTorch включает такие натренированные сети для различных архитектур (https://pytorch.org/docs/stable/torchvision/models.html)  
Мы будем использовать ResNet18.

Для начала посмотрим, что выдает уже натренированная сеть на наших картинках. То есть, посмотрим к какому из 1000 классов их отнесет сеть.

Запустите модель на 10 случайных картинках из датасета и выведите их вместе с классами с наибольшей вероятностью.  
В коде уже есть код, который формирует соответствие между индексами в выходном векторе и классами ImageNet.

In [ ]:
  import json
  !wget "https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json"
  class_idx = json.load(open("imagenet_class_index.json"))  

In [ ]:
# Thanks to https://discuss.pytorch.org/t/imagenet-classes/4923/2
def load_imagenet_classes():
    classes_json = urllib.request.urlopen('https://s3.amazonaws.com/deep-learning-models/image-models/imagenet_class_index.json').read()
    classes = json.loads(classes_json)
    
    # TODO: Process it to return dict of class index to name
    return { int(k): v[-1] for k, v in classes.items()}

  

def print_labels(class_idx, out, num_labels=10):
  idx2label = [class_idx[str(k)][1] for k in range(len(class_idx))]
  res=[]
  #print("top10", out[0].sort(descending = True, dim=-1)[1].detach().numpy()[:num_labels])
  for idx in out[0].sort(descending = True, dim=0)[1].detach().numpy()[:num_labels]:
    res.append(idx2label[idx])
  return res


model = models.resnet18(pretrained=True)
model.cuda()

indices = list(range(len(train_dataset)))
#np.random.seed(42)
np.random.shuffle(indices)
ind10 = indices[:10]

sampler10 = SubsetRandomSampler(ind10)


first_try_loader = torch.utils.data.DataLoader(train_dataset, batch_size=1, 
                                           sampler=sampler10)
#print(list(first_try_loader, val_loader))
labels=[]


for i, (x, y,_) in enumerate(first_try_loader):
  x_gpu = x.to(device)
  with torch.no_grad():
    prediction = model(x_gpu) 
    #print(prediction[0].sort()[0])
  #plt.title("Resnet 18 labels:\n %s" % print_labels(class_idx, prediction, 3))
  #plt.imshow(train_dataset[ind10[i]])
  labels.append(print_labels(class_idx, prediction.cpu(), 3))
  
visualize_samples(dataset, ind10, "resnet18", 10, labels)
#print(labels)
#visualize_samples(orig_dataset, indices, "Samples")
# TODO: Run this model on 10 random images of your dataset and visualize what it predicts

# Перенос обучения (transfer learning) - тренировать только последний слой

Существует несколько вариантов переноса обучения, мы попробуем основные.  
Первый вариант - заменить последний слой на новый и тренировать только его, заморозив остальные.

In [ ]:
import torch.nn as nn
import torch.optim as optim

model_st1 = models.resnet18(pretrained=True)
# TODO: Freeze all the layers of this model and add a new output layer
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

for param in model_st1.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_st1.fc.in_features
model_st1.fc = nn.Linear(num_ftrs, 2)

model_st1 = model_st1.to(device)

parameters = model_st1.fc.parameters()   # Fill the right thing here!

loss = nn.CrossEntropyLoss()
optimizer = optim.SGD( parameters, lr=0.001, momentum=0.9)
loss_history, train_history, val_history = train_model(model_st1, train_loader, val_loader, loss, optimizer, None, 5)

# Перенос обучения (transfer learning) - тренировать всю модель

Второй вариант - точно так же заменить последний слой на новый и обучать всю модель целиком.



In [68]:
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

model_st2 = models.resnet18(pretrained=True)
# TODO: Add a new output layer and train the whole model
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
num_ftrs = model_st2.fc.in_features
model_st2.fc = nn.Linear(num_ftrs, 2)

model_st2 = model_st2.to(device)

parameters = model_st2.fc.parameters()   # Fill the right thing here!

loss = nn.CrossEntropyLoss()
#optimizer = optim.SGD( parameters, lr=0.001, momentum=0.9)
optimizer = optim.Adam(parameters, lr=0.001, weight_decay = 1e-4)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
#exp_lr_scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 1, factor=0.2)

loss_history, train_history, val_history = train_model(model_st2, train_loader, val_loader, 
                                                       loss, optimizer, exp_lr_scheduler, 15)

Epoch: 0, Average loss: 0.428577, Train accuracy: 0.799891, Val accuracy: 0.900000, Val F1: 0.795014
Epoch: 1, Average loss: 0.357152, Train accuracy: 0.840348, Val accuracy: 0.918478, Val F1: 0.817539
Epoch: 2, Average loss: 0.330430, Train accuracy: 0.854195, Val accuracy: 0.920652, Val F1: 0.824536
Epoch: 3, Average loss: 0.331509, Train accuracy: 0.855281, Val accuracy: 0.919565, Val F1: 0.831884
Epoch: 4, Average loss: 0.321927, Train accuracy: 0.856096, Val accuracy: 0.913043, Val F1: 0.835260
Epoch: 5, Average loss: 0.320681, Train accuracy: 0.861797, Val accuracy: 0.920652, Val F1: 0.840708
Epoch: 6, Average loss: 0.308599, Train accuracy: 0.874016, Val accuracy: 0.926087, Val F1: 0.853767
Epoch: 7, Average loss: 0.322082, Train accuracy: 0.856096, Val accuracy: 0.927174, Val F1: 0.834553
Epoch: 8, Average loss: 0.310126, Train accuracy: 0.864513, Val accuracy: 0.927174, Val F1: 0.830904
Epoch: 9, Average loss: 0.314188, Train accuracy: 0.863427, Val accuracy: 0.925000, Val F1:

In [69]:
loss_history, train_history, val_history = train_model(model_st2, train_loader, val_loader, 
                                                       loss, optimizer, exp_lr_scheduler, 15)

Epoch: 0, Average loss: 0.303269, Train accuracy: 0.877545, Val accuracy: 0.938043, Val F1: 0.851632
Epoch: 1, Average loss: 0.305129, Train accuracy: 0.869400, Val accuracy: 0.935870, Val F1: 0.864865
Epoch: 2, Average loss: 0.307165, Train accuracy: 0.868857, Val accuracy: 0.927174, Val F1: 0.852941
Epoch: 3, Average loss: 0.301009, Train accuracy: 0.868585, Val accuracy: 0.931522, Val F1: 0.847858
Epoch: 4, Average loss: 0.312907, Train accuracy: 0.868585, Val accuracy: 0.900000, Val F1: 0.844444
Epoch: 5, Average loss: 0.307143, Train accuracy: 0.869128, Val accuracy: 0.932609, Val F1: 0.840708
Epoch: 6, Average loss: 0.312608, Train accuracy: 0.865056, Val accuracy: 0.910870, Val F1: 0.854599
Epoch: 7, Average loss: 0.328310, Train accuracy: 0.866142, Val accuracy: 0.939130, Val F1: 0.847407
Epoch: 8, Average loss: 0.298355, Train accuracy: 0.877817, Val accuracy: 0.932609, Val F1: 0.854573
Epoch: 9, Average loss: 0.308768, Train accuracy: 0.872658, Val accuracy: 0.940217, Val F1:

In [70]:
torch.save(model_st2.state_dict(), "model_st2.weights")

# Перенос обучения (transfer learning) - разные скорости обучения для разных слоев

И наконец последний вариант, который мы рассмотрим - использовать разные скорости обучения для новых и старых слоев

In [62]:
import torch.nn as nn
import torch.optim as optim

model_st3 = models.resnet18(pretrained=True)
# TODO: Add a new output layer
# Train new layer with learning speed 0.001 and old layers with 0.0001
# https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

num_ftrs = model_st3.fc.in_features
model_st3.fc = nn.Linear(num_ftrs, 2)

model_st3 = model_st3.to(device)

#parameters = model_st3.fc.parameters()   # Fill the right thing here!

loss = nn.CrossEntropyLoss()

optimizer = optim.Adam([
                {'params': model_st3.fc.parameters(), 'lr': 1e-2, 'weight_decay' : 1e-3}
            ], lr=1e-3, weight_decay = 1e-4)
exp_lr_scheduler = ReduceLROnPlateau(optimizer, 'min', patience = 3, factor=0.1)
loss_history, train_history, val_history = train_model(model_st3, train_loader, val_loader, 
                                                       loss, optimizer, exp_lr_scheduler, 20)




Epoch: 0, Average loss: 0.504205, Train accuracy: 0.805593, Val accuracy: 0.905435, Val F1: 0.835366
Epoch: 1, Average loss: 0.449793, Train accuracy: 0.822156, Val accuracy: 0.815217, Val F1: 0.829047
Epoch: 2, Average loss: 0.452058, Train accuracy: 0.838718, Val accuracy: 0.881522, Val F1: 0.841782
Epoch: 3, Average loss: 0.397116, Train accuracy: 0.850122, Val accuracy: 0.920652, Val F1: 0.860681
Epoch: 4, Average loss: 0.412344, Train accuracy: 0.845235, Val accuracy: 0.915217, Val F1: 0.858476
Epoch: 5, Average loss: 0.454904, Train accuracy: 0.835189, Val accuracy: 0.918478, Val F1: 0.852615
Epoch: 6, Average loss: 0.468482, Train accuracy: 0.836818, Val accuracy: 0.666304, Val F1: 0.852255
Epoch: 7, Average loss: 0.798230, Train accuracy: 0.806136, Val accuracy: 0.904348, Val F1: 0.839744
Epoch: 8, Average loss: 0.471833, Train accuracy: 0.845235, Val accuracy: 0.909783, Val F1: 0.840764
Epoch: 9, Average loss: 0.402046, Train accuracy: 0.852023, Val accuracy: 0.816304, Val F1:

# Визуализируем метрики и ошибки модели

Попробуем посмотреть, где модель ошибается - визуализируем ложные срабатывания (false positives) и ложноотрицательные срабатывания (false negatives).

Для этого мы прогоним модель через все примеры и сравним ее с истинными метками (ground truth).

In [ ]:


predictions, gt = evaluate_model(model_st2, train_dataset, val_indices)
print(predictions, "\n", gt)

И теперь можно визуализировать false positives и false negatives.

In [ ]:
# TODO: Compute indices of the false positives on the validation set.
# Note those have to be indices of the original dataset
diff = (np.array(predictions) - np.array(gt))

false_positive_indices = np.array(val_indices)[diff == 1]
visualize_samples(orig_dataset, false_positive_indices, "False positives")

# TODO: Compute indices of the false negatives on the validation set.
# Note those have to be indices of the original dataset
false_negatives_indices = np.array(val_indices)[diff == -1]
visualize_samples(orig_dataset, false_negatives_indices, "False negatives")


In [ ]:
precision, recall, f1 = binary_classification_metrics(predictions, gt)
print("F1: %4.3f, P: %4.3f, R: %4.3f" % (f1, precision, recall))

# Что будет в конце вы уже поняли

Натренируйте лучшую модель на основе `resnet18`, меняя только процесс тренировки.
Выбирайте лучшую модель по F1 score.

Как всегда, не забываем:
- побольше агментаций!
- перебор гиперпараметров
- различные оптимизаторы
- какие слои тюнить
- learning rate annealing
- на какой эпохе останавливаться

Наша цель - довести F1 score на validation set до значения, большего **0.93**.

In [ ]:
# TODO: Train your best model!
best_model = model_st3

In [ ]:
# Let's check how it performs on validation set!
predictions1, ground_truth = evaluate_model(model_st3, val_dataset, val_indices)
precision, recall, f1 = binary_classification_metrics(predictions1, ground_truth)
print("F1: %4.3f, P: %4.3f, R: %4.3f" % (f1, precision, recall))

# TODO: Visualize training curve for the best model

## Визуализируйте ошибки лучшей модели

In [ ]:
# TODO Visualize false positives and false negatives of the best model on the validation set

# Необязательное задание с большой звездочкой

Поучавствуйте в Kaggle In-Class Hot Dog Recognition Challenge!  
Это соревнование сделано специально для курса и в нем учавствуют только те, кто проходит курс.

В нем участники соревнуются в качестве натренированных моделей, загружая на сайт предсказания своих моделей на тестовой выборке. Разметка тестовой выборке участникам недоступна.
Более подробно о правилах соревнования ниже.

Те, кто проходят курс лично, за высокое место в соревновании получат дополнительные баллы.

Здесь уже можно использовать и другие базовые архитектуры кроме `resnet18`, и ансамбли, и другие трюки тренировки моделей.

Вот ссылка на соревнование:
https://www.kaggle.com/c/hotdogornot

In [ ]:
image_id = []
predictions = []
model.eval()
for x,_,id_img in test_loader:
    # TODO : Напишите код для предсказания меток (1 = есть хотдог, 0 = хотдога нет)
    # Код должен возвратить список из id картинки и метку predictions
    # image id - это название файла картинки, например '10000.jpg'
    pass

In [ ]:
# Так можно создать csv файл, чтобы затем загрузить его на kaggle
# Ожидаемый формат csv-файла:
# image_id,label
# 10000.jpg,1
# 10001.jpg,1
# 10002.jpg,0
# 10003.jpg,1
# 10004.jpg,0

with open('subm.csv', 'w') as submissionFile:
    writer = csv.writer(submissionFile)
    writer.writerow(['image_id', 'label'])
    writer.writerows(zip(image_id,predictions))

In [ ]:
# А так можно скачать файл с Google Colab
files.download('subm.csv') 

### Небольшое введение в Kaggle для тех, кто не слышал об этой платформе раньше

В основе своей Kaggle - это платформа для проведения соревнований по машинному обучению. Появилась она в 2010 и, пожалуй, стала самой популярной и известной из всех существующих площадок по машинному обучению. Надо сказать, что Kaggle - это не только соревнования, но и сообщество людей, увлеченных машинным обучением. А судя по Википедии, в 2017 году отметка зарегистрированных пользователей перевалила за миллион. Есть там и обучающие материалы, возможность задавать вопросы, делиться кодом и идеями - просто мечта. 

### Как проходят соревнования? 
Обычно участники скачивают данные для обучения моделей (train data), чтобы затем делать предсказания на тестовых данных (test data). Обучающая выборка содержит как сами данные, так и правильные метки (значения зависимой переменной), чтобы можно было обучить модель. Но тестовые данные ответа не содержат - и нашей целью является предсказание меток по имеющимся данным. Файл с ответами для каждого наблюдения из тестовой выборки загружается на Kaggle и оценивается в соответствии с выбранной метрикой соревнования, а результат является публичным и показывается в общей таблице (ее называют еще лидербордом - leaderboard) - чтобы появилось желание посоревноваться и создать еще более сильную модель. В "настоящих" соревнованиях, которые проходят на Kaggle, есть и денежное вознаграждение для тех участников, кто занимает первые места на лидерборде. Например, в [этом](https://www.kaggle.com/c/zillow-prize-1#description) соревновании, человек, занявший первое место, получил около 1 000 000 долларов. 

Тестовые данные делятся случайным образом в некоторой пропорции. И пока соревнование идет, на лидерборде показываются очки и рейтинг участников только по одной части (Public Leaderboard). А вот когда соревнование заканчивается, то рейтинг участников составляется по второй части тестовых данных (Private Leaderboard). И часто можно видеть, как люди занимавшие первые места на публичной части тестовых данных, оказываются далеко не первыми на закрытой части тестовых данных. Зачем это сделано? Есть несколько причин, но, пожалуй, самой фундаментальной является идея недообучения-переобучения. Всегда возможно, что наша модель настроилась на конкретную выборку, но как она поведет себя на тех данных, которые еще не видела? Разбиение тестовых данных на публичную и скрытую части сделано для того, чтобы отобрать модели, которые имеют большую обобщающую способность. Одним из лозунгов участников соревнований стал "Доверяйте своей локальной кросс-валидации" (Trust your CV!). Есть очень большой соблазн оценивать свою модель по публичной части лидерборда, но лучшей стратегией будет выбирать ту модель, которая дает лучшую метрику на кросс-валидации на обучающей выборке. 

В нашем соревновании публичная часть лидерборда составляет 30%, а скрытая 70%. Вы можете делать до двух попыток в день, а оцениваться попытки будут по F1-мере. Удачи и доверяйте своей локальной валидации! В конце соревнования у вас будет возможность выбрать 2 из всех совершенных попыток - лучшая из этих двух и будет засчитана вам на скрытой части тестовых данных.